In [18]:
import folium
import pandas as pd
import json
from folium import plugins
import geocoder
from geopy.distance import great_circle

In [19]:
with open('./전국_정신건강관련기관_위치정보.json', 'r',encoding='utf-8') as f:
    loc_data = json.load(f)
    
loc_arr = loc_data['data']
loc_arr

df = pd.DataFrame(loc_arr)


df['위도'] = pd.to_numeric(df['위도'], errors='coerce')
df['경도'] = pd.to_numeric(df['경도'], errors='coerce')

df.loc[df['기관구분']=='정신병원', '기관구분'] = '병원'

df.index = range(len(df))
df

,index,공공/민간,기관구분,기관명,주소,위도,경도,전화번호,홈페이지
0,0,공공,상담소,(주)마음꼭 차심리상담센터,경기도 성남시 분당구 야탑동 351 기억력센터 2층 (야탑로 59 분당차병원),37.410250,127.125250,031-780-5437,None
1,1,공공,상담소,(주)마음연구소 부설 마음iN심리상담센터,서울 종로구 종로3길 34 6층,37.571875,126.979481,02-722-2883,None
2,2,공공,상담소,가능성연구소,부산시 해운대구 세실로79 부산해운대대교사옥 6층,35.171647,129.176440,051-704-8610,None
3,3,공공,상담소,가톨릭대학교 은평성모병원,서울 은평구 진관동 93-6,37.633251,126.916547,02-1811-7755,None
4,4,공공,상담소,강원대학교병원 정신건강의학과,강원도 춘천시 백령로 156 강원대학교병원 정신건강의학과,37.874701,127.744801,033-258-2437,None
...,...,...,...,...,...,...,...,...,...
4327,4706,민간,병원,실버한방병원,서울특별시 금천구 독산로 362 (독산동),37.479089,126.904289,02-857-2001,None
4328,4707,민간,병원,의료법인 영명의료재단 혜민한방병원,"경기도 시흥시 군자로 526 도일그랜드프라자 302,401,501호 (거모동)",37.343354,126.785183,031-504-7272,None
4329,4708,민간,병원,한도한방병원,경기도 의정부시 입석로 36 (가능동),37.755883,127.021652,031-877-6081,None
4330,4709,민간,병원,혜민한방병원,충청남도 태안군 태안읍 정주내1길 13-0 혜민한방병원,36.745075,126.304532,041-400-7171,None


In [6]:
res = df.drop_duplicates("기관구분", keep="first")
res

,index,공공/민간,기관구분,기관명,주소,위도,경도,전화번호,홈페이지
0,0,공공,상담소,(주)마음꼭 차심리상담센터,경기도 성남시 분당구 야탑동 351 기억력센터 2층 (야탑로 59 분당차병원),37.410250,127.125250,031-780-5437,None
353,368,공공,센터,(재)천주의성요한병원,"광주광역시 북구 태봉로 32, 유동 115-1",35.160333,126.903603,null,http://www.yohanhos.or.kr/
355,370,민간,시설,가득한집,"서울특별시 금천구 시흥대로 133-7, 301호(시흥동)",37.445994,126.902780,null,www.homelessness.or.kr
359,374,공공,보건소,가평군보건소,경기도 가평군 가평읍 가화로 155-18,37.833503,127.510601,null,https://www.gp.go.kr/
1144,1370,민간,병원,가톨릭대학교인천성모병원,인천광역시 부평구 동수로 56- (부평동),37.484831,126.724899,032-1544-9004,http://www.cmcism.or.kr/


In [20]:
class MappingByCoord:

    
    def __init__(self, df, lat, lng, dist = 5):

        self.df = df
        self.lat = lat
        self.lng = lng
        self.dist = dist
        
    def setRectangler(self):

        lat_min = self.lat - 0.01 * self.dist
        lat_max = self.lat + 0.01 * self.dist

        lng_min = self.lng - 0.015 * self.dist
        lng_max = self.lng + 0.015 * self.dist

        self.points = [[lat_min, lng_min], [lat_max, lng_max]]

        result = self.df.loc[
            (self.df['위도'] > lat_min) &
            (self.df['위도'] < lat_max) &
            (self.df['경도'] > lng_min) &
            (self.df['경도'] < lng_max)
        ]
        result.index = range(len(result))

        return result    
        

    def setCircle(self):

        tmp = self.setRectangler()

        center = (self.lat, self.lng)

        result = pd.DataFrame()

        for index, row in tmp.iterrows():
            point = (row['위도'], row['경도'])
            d = great_circle(center, point).kilometers
            if d <= self.dist:
                result = pd.concat([result, tmp.iloc[index, :].to_frame().T])
            
        result.index = range(len(result))
            
        return result
        
    def MappingInRectangler(self, df):


        m = folium.Map(location=[self.lat, self.lng], zoom_start=14)
        

        for idx, row in df.iterrows():

            lat_now = row['위도']
            lng_now = row['경도']

            folium.Marker(location=[lat_now, lng_now],
                          radius=15,
                          tooltip=row['기관명']).add_to(m)

        folium.Rectangle(bounds=self.points,
                         color='#ff7800',
                         fill=True,
                         fill_color='#ffff00',
                         fill_opacity=0.2).add_to(m)

        return m 
    
    def MappingInCircle(self, df):

        m = folium.Map(location=[self.lat, self.lng], width='70%', height='100%')

        for idx, row in df.iterrows():

            lat_now = row['위도']
            lng_now = row['경도']

            folium.Marker(location=[lat_now, lng_now],
                          radius=15,
                          tooltip=row['기관명']).add_to(m)
            
                
        folium.Circle(radius=dist * 1000,
                      location=[lat, lng],
                      color="#ff7800",
                      fill_color='#ffff00',
                      fill_opacity=0.2
                      ).add_to(m)

        return m
    


In [25]:
# def makeMap(request):
user_loc = geocoder.ip('me')
mylocation = user_loc.latlng
    
lat = mylocation[0]
lng = mylocation[1]
dist = 5


mbc = MappingByCoord(df, lat, lng, dist)

result_rectangler = mbc.MappingInRectangler()

result_circle = mbc.MappingInCircle()
    

print(f"""
{"="*50}
중심 위도: {mbc.lat}
중심 경도: {mbc.lng}
기준 거리: {mbc.dist} km
사각 범위 내 데이터 필터링 결과: {len(result_rectangler):,} 건
반경 범위 내 데이터 필터링 결과: {len(result_radius):,} 건
{"="*50}
""")

    
#mymap = mbc.MappingInCircle(result_circle)
# maps = mymap._repr_html_()
# return render(request, 'map_show.html', {'mymap': maps})

#mymap

TypeError: MappingInRectangler() missing 1 required positional argument: 'df'

In [ ]:
            div = ['상담소', '센터', '시설', '보건소', '병원']
            
            mcg = folium.plugins.MarkerCluster(control=False)
            m.add_child(mcg)
            상담소 = folium.plugins.FeatureGroupSubGroup(mcg, "상담소")
            센터 = folium.plugins.FeatureGroupSubGroup(mcg, "센터")
            시설 = folium.plugins.FeatureGroupSubGroup(mcg, "시설")
            병원 = folium.plugins.FeatureGroupSubGroup(mcg, "병원")
            보건소 = folium.plugins.FeatureGroupSubGroup(mcg, "보건소")
            
            if row['기관구분'] = div[0]:
                folium.Marker([lat_now, lng_now], icon=folium.Icon(color='pink')).add_to('상담소')
                m.add_child(상담소)
            elif row['기관구분'] = div[1]:
                folium.Marker([lat_now, lng_now], icon=folium.Icon(color='green')).add_to('센터')
                m.add_child(센터)
            elif row['기관구분'] = div[2]:
                folium.Marker([lat_now, lng_now], icon=folium.Icon(color='blue')).add_to('시설')
                m.add_child(시설)
            elif row['기관구분'] = div[3]:
                folium.Marker([lat_now, lng_now], icon=folium.Icon(color='purple')).add_to('병원')
                m.add_child(병원)
            else:
                folium.Marker([lat_now, lng_now], icon=folium.Icon(color='ornage')).add_to('보건소')
                m.add_child(보건소)

In [11]:
class MappingByCoord:

    
    def __init__(self, df, lat, lng, dist = 5):

        self.df = df
        self.lat = lat
        self.lng = lng
        self.dist = dist
        
    def setRectangler(self):

        lat_min = self.lat - 0.01 * self.dist
        lat_max = self.lat + 0.01 * self.dist

        lng_min = self.lng - 0.015 * self.dist
        lng_max = self.lng + 0.015 * self.dist

        self.points = [[lat_min, lng_min], [lat_max, lng_max]]

        result = self.df.loc[
            (self.df['위도'] > lat_min) &
            (self.df['위도'] < lat_max) &
            (self.df['경도'] > lng_min) &
            (self.df['경도'] < lng_max)
        ]
        result.index = range(len(result))

        return result    
        

    def setCircle(self):

        tmp = self.setRectangler()

        center = (self.lat, self.lng)

        result = pd.DataFrame()

        for index, row in tmp.iterrows():
            point = (row['위도'], row['경도'])
            d = great_circle(center, point).kilometers
            if d <= self.dist:
                result = pd.concat([result, tmp.iloc[index, :].to_frame().T])
            
        result.index = range(len(result))
            
        return result
        
    def MappingInRectangler(self, df):


        m = folium.Map(location=[self.lat, self.lng], zoom_start=14)
        

        for idx, row in df.iterrows():

            lat_now = row['위도']
            lng_now = row['경도']

            folium.Marker(location=[lat_now, lng_now],
                          radius=15,
                          tooltip=row['기관명']).add_to(m)

        folium.Rectangle(bounds=self.points,
                         color='#ff7800',
                         fill=True,
                         fill_color='#ffff00',
                         fill_opacity=0.2).add_to(m)

        return m 
    
    def MappingInCircle(self, df):

        m = folium.Map(location=[self.lat, self.lng], width='70%', height='100%')

        for idx, row in df.iterrows():

            lat_now = row['위도']
            lng_now = row['경도']

            folium.Marker(location=[lat_now, lng_now],
                          radius=15,
                          tooltip=row['기관명']).add_to(m)
            div = ['상담소', '센터', '시설', '보건소', '병원']
            
            mcg = folium.plugins.MarkerCluster(control=False)
            m.add_child(mcg)
            상담소 = folium.plugins.FeatureGroupSubGroup(mcg, "상담소")
            센터 = folium.plugins.FeatureGroupSubGroup(mcg, "센터")
            시설 = folium.plugins.FeatureGroupSubGroup(mcg, "시설")
            병원 = folium.plugins.FeatureGroupSubGroup(mcg, "병원")
            보건소 = folium.plugins.FeatureGroupSubGroup(mcg, "보건소")
            
            if row['기관구분'] == div[0]:
                folium.Marker([lat_now, lng_now], icon=folium.Icon(color='pink')).add_to('상담소')
                m.add_child(상담소)
            elif row['기관구분'] == div[1]:
                folium.Marker([lat_now, lng_now], icon=folium.Icon(color='green')).add_to('센터')
                m.add_child(센터)
            elif row['기관구분'] == div[2]:
                folium.Marker([lat_now, lng_now], icon=folium.Icon(color='blue')).add_to('시설')
                m.add_child(시설)
            elif row['기관구분'] ==div[3]:
                folium.Marker([lat_now, lng_now], icon=folium.Icon(color='purple')).add_to('병원')
                m.add_child(병원)
            else:
                folium.Marker([lat_now, lng_now], icon=folium.Icon(color='ornage')).add_to('보건소')
                m.add_child(보건소)
            
                
        folium.Circle(radius=dist * 1000,
                      location=[lat, lng],
                      color="#ff7800",
                      fill_color='#ffff00',
                      fill_opacity=0.2
                      ).add_to(m)

        return m
    


In [12]:
user_loc = geocoder.ip('me')
mylocation = user_loc.latlng
    
lat = mylocation[0]
lng = mylocation[1]
dist = 5


mbc = MappingByCoord(df, lat, lng, dist)

result_radius = mbc.MappingInCircle()
    
mymap = mbc.MappingInCircle(result_radius)
folium.Marker(location=mylocation, popup='현재 나의 위치', icon=folium.Icon(color='red', icon='star')).add_to(mymap)
plugins.LocateControl().add_to(mymap)
    
    
#maps = mymap._repr_html_()
# return render(request, 'map_show.html', {'mymap': maps})

mymap

TypeError: MappingInCircle() missing 1 required positional argument: 'df'

In [ ]:
class MappingByCoord:

    
    def __init__(self, df, lat, lng, dist = 5):

        self.df = df
        self.lat = lat
        self.lng = lng
        self.dist = dist
        
    def setRectangler(self):

        lat_min = self.lat - 0.01 * self.dist
        lat_max = self.lat + 0.01 * self.dist

        lng_min = self.lng - 0.015 * self.dist
        lng_max = self.lng + 0.015 * self.dist

        self.points = [[lat_min, lng_min], [lat_max, lng_max]]

        result = self.df.loc[
            (self.df['위도'] > lat_min) &
            (self.df['위도'] < lat_max) &
            (self.df['경도'] > lng_min) &
            (self.df['경도'] < lng_max)
        ]
        result.index = range(len(result))

        return result    
        

    def setCircle(self):

        tmp = self.setRectangler()

        center = (self.lat, self.lng)

        result = pd.DataFrame()

        for index, row in tmp.iterrows():
            point = (row['위도'], row['경도'])
            d = great_circle(center, point).kilometers
            if d <= self.dist:
                result = pd.concat([result, tmp.iloc[index, :].to_frame().T])
            
        result.index = range(len(result))
            
        return result
        
    def MappingInRectangler(self, df):


        m = folium.Map(location=[self.lat, self.lng], zoom_start=14)
        

        for idx, row in df.iterrows():

            lat_now = row['위도']
            lng_now = row['경도']

            folium.Marker(location=[lat_now, lng_now],
                          radius=15,
                          tooltip=row['기관명']).add_to(m)

        folium.Rectangle(bounds=self.points,
                         color='#ff7800',
                         fill=True,
                         fill_color='#ffff00',
                         fill_opacity=0.2).add_to(m)

        return m 
    
    def MappingInCircle(self, df):

        m = folium.Map(location=[self.lat, self.lng], width='70%', height='100%')

        for idx, row in df.iterrows():

            lat_now = row['위도']
            lng_now = row['경도']

            folium.Marker(location=[lat_now, lng_now],
                          radius=15,
                          tooltip=row['기관명']).add_to(m)
            
                
        folium.Circle(radius=dist * 1000,
                      location=[lat, lng],
                      color="#ff7800",
                      fill_color='#ffff00',
                      fill_opacity=0.2
                      ).add_to(m)

        return m